クロスデバイス　Valis拡張との併用

In [168]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

5850	株式会社ラビット・カーネットワーク

In [169]:
today_yymmdd = date.today().strftime("%y%m%d")

In [170]:
print(today_yymmdd)

190307


In [172]:
#デバイス
seed_sp_device = 'iOS','Android'
xd_sp_device = 'iphone','android_phone','windows_phone'

#ターゲット
advertiser_id = 5850
retargeting_group_id = 81535

#grp = 'cvg'
grp = 'rtg'

In [171]:
#クロスデバイステーブル
xd_table = 'segment.drawbridge_device_1901'

PCにユーザーをSPクロスデバイスし、これを種にV拡張

%%slack_notify @yusuke_akada

sd = 'pc'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,conversion_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and conversion_group_id in {conversion_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family not in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_cvg_{conversion_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"


df_seed = df.drop(['conversion_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

df.shape[0]

df.head()

%%slack_notify @yusuke_akada
#XD拡張

xd = 'sp'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_3077other_{today_yymmdd}"
#uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{conversion_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False, header=False)

df_fix.shape[0]

PCユーザーをSPにクロスデバイスし、これを種にV拡張

In [173]:
%%slack_notify @yusuke_akada

sd = 'pc'
grp = 'rtg'

#f-string
query = (
    f"""
    select
    distinct smn_uid
    ,advertiser_id
    ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
    ,retargeting_group_id
    FROM
    dsplog.adserver
    WHERE
    concat_ws('-',year,month,day) between to_date(days_add(now(),-30)) and to_date(days_add(now(),-1))
    and retargeting_group_id = {retargeting_group_id}
    and advertiser_id = {advertiser_id}
    and user_agent_os_family not in {seed_sp_device}
    and smn_uid is not null
    """
    )

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df = ir.sql_to_pandas(query)
        ir.upload_dataframe(f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}", df, ['bigint','bigint','string','bigint'])
        

seed_table = f"dm_tmp.ya_seed_{sd}_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}"

df_seed = df.drop(['retargeting_group_id','device', 'advertiser_id'], axis=1)
df_seed.to_csv(f'{seed_table}.tsv', sep='\t', index=False, header=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [174]:
df.shape[0]

3415

In [175]:
df.head()

,smn_uid,advertiser_id,device,retargeting_group_id
0,186000000084626,5850,PC,81535
1,1269000003787604,5850,PC,81535
2,2180000334411726,5850,PC,81535
3,2188000100236939,5850,PC,81535
4,2062000139634807,5850,PC,81535


In [176]:
%%slack_notify @yusuke_akada
#XD拡張

xd = 'sp'

uid_table = f"dm_tmp.ya_xd_{xd}_advid_{advertiser_id}_{grp}_{retargeting_group_id}_{today_yymmdd}"

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {seed_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in {xd_sp_device}  
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    #ir.upload_dataframe(uid_table, df_fix, ['bigint']) #Impala upは不要かも
    #テーブル削除
    ir.drop_tables([seed_table])
    
df_fix.to_csv(f'{uid_table}.tsv', sep='\t', index=False)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [177]:
df_fix.shape[0]

9046